<a href="https://colab.research.google.com/github/rprimi/b5_nlp/blob/main/m2_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pacakges <- c("tidyverse", "keras", "psych")
install.packages(pacakges)

install.packages("readr")
install.packages("zip")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [10]:
library(tidyverse)
library(keras)
library(psych)
library(readr)
library(zip)



Attaching package: ‘zip’


The following objects are masked from ‘package:utils’:

    unzip, zip




In [4]:
load("base_bfi_nlp.RData")

In [ ]:
samples <- db_textos_splited$texto_dividido

tokenizer <- text_tokenizer(
    lower = TRUE )  %>%
    fit_text_tokenizer(samples)

word_index <- tokenizer$word_index

sequences <- texts_to_sequences(tokenizer, samples)

cat("Found", length(word_index), "unique tokens.\n")


In [ ]:
library(zip)

# Download the zip file
download.file("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s600.zip", destfile = "file.zip")

# Read the CSV file from the zip file
glove_pt_600 <- read.zip("file.zip", "file.csv")



In [14]:
 load("nilc_wv_glove_600d.RData")
 ls()


 glove_pt_600 = data.table::fread(
    file = "~/Downloads/glove_s600.txt",
    encoding = 'UTF-8',
    # nrows = 100,
    skip = 1
  )

  names(nilc_wv_glove_600d.RData)

  db_textos_embeddings <- db_textos_splited %>%
    unnest_tokens(word, texto_dividido) %>%
    left_join(glove_pt_600, by =c("word"="V1")) %>%
    select(-word) %>%
    group_by(id, id_divisao) %>%
    dplyr::summarise(
    across(V2:V601, mean, na.rm=TRUE)
)

ERROR: ignored

In [ ]:

# Setting the embedding matrix

   vocabulary <- tibble(
     word = names(word_index),
     index = word_index
   )

   vocabulary <- vocabulary %>%
    left_join(glove_pt_600, by =c("word"="V1"))

  names(vocabulary)

# Function adapted from Chalot book
  prepare_embedding_matrix <- function(num_words, EMBEDDING_DIM, word_index) {
      MAX_NUM_WORDS = num_words
      embedding_matrix <- matrix(0L, nrow = num_words+1, ncol = EMBEDDING_DIM)
      for (word in names(word_index)) {
        index <- word_index[[word]]
        if (index >= MAX_NUM_WORDS)
          next
        embedding_vector <- as.numeric(vocabulary[vocabulary$word == word, 3:602])
        if (!is.null(embedding_vector)) {
          # words not found in embedding index will be all-zeros.
          embedding_matrix[index+1,] <- embedding_vector
        }
      }
      embedding_matrix
    }

# Creates embedding matrix
     embedding_matrix <- prepare_embedding_matrix(
         num_words = 66128,
         EMBEDDING_DIM = 600,
         word_index = word_index)

# Fills empty rows with random number
    rnd <- runif(n=sum(is.na(embedding_matrix)), min = 0, max = .04)
    embedding_matrix[is.na(embedding_matrix)] <- rnd

# Test it
   table(is.na(embedding_matrix))

# See shape of embedding matrix and first row (needs to be zero, don't know why)
# https://github.com/rstudio/keras/issues/302)
   dim(embedding_matrix)
   embedding_matrix[1, ]


In [ ]:

    hist(map_dbl(sequences, length))

# Shuffle data
    set.seed(8)

    indices <- sample(1:length(sequences))

    prop_train <- .82
    maxlen <-250

    train_indices <- 1: round(prop_train*length(sequences), 0)
    test_indices <-  (round(prop_train*length(sequences), 0)+1) : length(sequences)


    x_train <- pad_sequences(sequences[indices[train_indices]], maxlen = maxlen)
    x_test <- pad_sequences(sequences[indices[test_indices]], maxlen = maxlen)

    db_ys <- db_textos_splited[ , "id"] %>%
      as.data.frame() %>%
      set_names("id") %>%
      mutate(id = as.numeric(id)) %>%
      left_join( {db_bfi %>% select(id, O_rec:N_vlti_rec)}, by = "id" )


    y_train <- as.matrix(db_ys[indices[train_indices] , 2:6])
    y_test <-  as.matrix(db_ys[indices[test_indices] , 2:6])

    dim(x_train)
    dim(x_test)

    dim(y_train)
    dim(y_test)


In [ ]:
  max_words = 66128
    embedding_dim = 600

# bidirectional lstm model

   model <- keras_model_sequential() %>%
     layer_embedding(
         input_dim = max_words + 1,
         output_dim = embedding_dim ,
         weights = list(embedding_matrix),
         input_length = maxlen,
         trainable = FALSE ) %>%
    bidirectional(layer_lstm(units = 64,recurrent_dropout = 0.5, dropout =.5)) %>%
    layer_dense(units = 5,  kernel_regularizer = regularizer_l2(0.001))

    summary(model)


In [ ]:

# for continouse y-variable
    model %>% compile(
        optimizer = "rmsprop",
        loss = "mse",
        metrics = c("mae")
    )
    history <- model %>% fit(
      x_train,
      y_train,
      epochs =9,
      batch_size = 100,
      validation_data = list(x_test, y_test)
    )

    plot(history)


In [ ]:

    results <-  model %>% evaluate(x_test, y_test)
    predictions <-  model %>% predict(x_test)
    results

    cbind(predictions, y_test) %>%  corr.test()

    ggplot(
        data = data.frame(
            cbind(y_test, y_pred = predictions[ , 1])
            ),
        aes(x = y_test, y = y_pred) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(method = "lm") +
        geom_smooth(color = "red")

   resp2[indices[test_indices], ] %>%
       cbind(predictions) %>%
       select(Código, Ma_measure, y_theta_z, predictions) %>%
       group_by(Código) %>%
       summarise_all(.funs = mean) %>%
       select(-Código) %>%
       ggplot(aes(x = Ma_measure, y = predictions) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(method = "lm") +
        geom_smooth(color = "red")


    resp2[indices[test_indices], ] %>%
       cbind(predictions) %>%
       select(Código, Ma_measure, y_theta_z, predictions) %>%
       group_by(Código) %>%
       summarise_all(.funs = mean) %>%
       select(-Código) %>%
       corr.test()


    results <- model %>% evaluate(x_train, y_train)
    predictions <- model %>% predict(x_train)
    cor(predictions, y_train)

    ggplot(
        data = data.frame(
            cbind(y_train, y_pred = predictions[ , 1])
            ),
        aes(x = y_train, y = y_pred) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(color = "orange") +
        geom_smooth(method = "lm")

